In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline 

In [2]:
df_train = pd.read_pickle('../input/feats/df_trn_feat0.pkl')

In [3]:
df_train.columns

Index(['uidx', 'iidx', 'iid', 'region_city_label', 'tit_len', 'desc_len',
       'activation_date', 'month', 'day', 'weekday', 'param_1', 'param_2',
       'param_3', 'user_type', 'parent_category_name', 'price',
       'category_name', 'image_top_1', 'ads_cnt_by_uid', 'ads_cnt_by_iid',
       'deal_probability'],
      dtype='object')

In [4]:
mask0 = df_train.deal_probability == 0
mask1 = df_train.deal_probability == 1
X_trn_deal_0 = df_train[mask0]
X_trn_deal_1 = df_train[mask1]

In [5]:
X_trn_deal_0.shape

(974618, 21)

In [6]:
X_trn_deal_1.shape

(10076, 21)

In [11]:
X_trn_deal_0[pd.isnull(X_trn_deal_0.price)]

(36751, 21)

deal probability = 0 --> 974,618 / 1.5 milion (over 50%)

In [17]:
import lightgbm as lgb

In [24]:
df_train['y_zero'] = mask0.astype('uint8')

In [30]:
selcols = list(df_train.columns)
drop_cols = ['deal_probability','activation_date','iid','y_zero']
selcols = [col for col in selcols if col not in drop_cols]
X_train = df_train[selcols].copy()
y_train = mask0.astype('uint8')


In [31]:
predictors = list(X_train.columns)

In [32]:
predictors

['uidx',
 'iidx',
 'region_city_label',
 'tit_len',
 'desc_len',
 'month',
 'day',
 'weekday',
 'param_1',
 'param_2',
 'param_3',
 'user_type',
 'parent_category_name',
 'price',
 'category_name',
 'image_top_1',
 'ads_cnt_by_uid',
 'ads_cnt_by_iid']

In [33]:
lgbm_params =  {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',    
    'max_depth': -1,
    'num_leaves': 33,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.8,
    # 'bagging_freq': 5,
    'learning_rate': 0.05,
    'verbose': 20
} 
predictors = list(X_train.columns)

categorical = ['month','day','weekday',
               'param_1','param_2','param_3',
               'category_name','parent_category_name',
               'user_type']




dtrain = lgb.Dataset(X_train, label=y_train, feature_name=predictors, categorical_feature=categorical)
cv_results = lgb.cv(lgbm_params, dtrain, 
                    num_boost_round=10000, 
                    nfold=4,
                    stratified=False,
                    early_stopping_rounds=10,
                    verbose_eval= 100)
print('done')

/home/ihong/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1005: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	cv_agg's auc: 0.806964 + 0.000552226
[200]	cv_agg's auc: 0.810522 + 0.000576355
[300]	cv_agg's auc: 0.812095 + 0.00058868
[400]	cv_agg's auc: 0.813173 + 0.000602578
[500]	cv_agg's auc: 0.813967 + 0.000567106
[600]	cv_agg's auc: 0.814497 + 0.000529122
[700]	cv_agg's auc: 0.814898 + 0.00054234
[800]	cv_agg's auc: 0.815214 + 0.000522802
[900]	cv_agg's auc: 0.815458 + 0.000469385
[1000]	cv_agg's auc: 0.815703 + 0.000461226
[1100]	cv_agg's auc: 0.815895 + 0.000449416
[1200]	cv_agg's auc: 0.816041 + 0.000431564
[1300]	cv_agg's auc: 0.816204 + 0.000456995
[1400]	cv_agg's auc: 0.816334 + 0.000470158
[1500]	cv_agg's auc: 0.81641 + 0.000463659
done


In [ ]:
dtrain = lgb.Dataset(X_train, label=y_train, feature_name=predictors, categorical_feature=categorical)
lgb_clf = lgb.train(lgbm_params,num_boost_round = 10000,
                        train_set = dtrain,
                        valid_sets = [dtrain],                        
                        early_stopping_rounds=10,
                        verbose_eval= 100)

/home/ihong/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1005: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 10 rounds.
[100]	training's auc: 0.808353
[200]	training's auc: 0.813483
[300]	training's auc: 0.816496
[400]	training's auc: 0.818714
[500]	training's auc: 0.820588
[600]	training's auc: 0.822098
[700]	training's auc: 0.823526
[800]	training's auc: 0.824729
[900]	training's auc: 0.825909
[1000]	training's auc: 0.826966
[1100]	training's auc: 0.827909
[1200]	training's auc: 0.828864
[1300]	training's auc: 0.829797
[1400]	training's auc: 0.830676
[1500]	training's auc: 0.831532
[1600]	training's auc: 0.832322
[1700]	training's auc: 0.833192
[1800]	training's auc: 0.833981
[1900]	training's auc: 0.834745
[2000]	training's auc: 0.835394
